In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor, XGBClassifier
import numpy as np
from scipy import stats
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv(r"D:\python\projects\car_price_prediction\Samochody Otomoto1")
df.head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor
0,21900,Toyota,RAV4,2003,218000,1794,Benzyna,125,Manualna,5,Niebieski
1,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor
2,32000,Mercedes-Benz,Klasa A,2013,110000,1595,Benzyna,122,Automatyczna,5,Inny kolor
3,34900,Toyota,AvensisGeneracjaIII (2009-),2012,325000,2231,Diesel,149,Manualna,5,Inny kolor
4,26500,Opel,AstraGeneracjaJ (2009-2019),2010,201000,1686,Diesel,110,Manualna,5,Srebrny


In [3]:
labelencoder = LabelEncoder()

In [4]:
for x in df["przebieg"]:
    if len(x) > 10:
        df = df[df["przebieg"] != x]
        
for x in df["cena"]:
    if len(x) > 10:
        df = df[df["cena"] != x]

In [5]:
for x in df["cena"]:
    try:
        x = float(x)
    except:
        df = df[df["cena"] != x]

In [6]:
for x in df["moc"]:
    try:
        x = float(x)
    except:
        df = df[df["moc"] != x]

In [7]:
for x in df["pojemnosc"]:
    try:
        x = float(x)
    except:
        df = df[df["pojemnosc"] != x]

In [8]:
df["cena"] = pd.to_numeric(df["cena"])
df["marka label"] = labelencoder.fit_transform(df["marka"])
df["model label"] = labelencoder.fit_transform(df["model"])
df["rocznik"] = pd.to_numeric(df["rocznik"])
df["przebieg"] = pd.to_numeric(df["przebieg"])
df["pojemnosc"] = pd.to_numeric(df["pojemnosc"])
df["rodzaj paliwa label"] = labelencoder.fit_transform(df["rodzaj paliwa"])
df["moc"] = pd.to_numeric(df["moc"])
df["skrzynia biegow label"] = labelencoder.fit_transform(df["skrzynia biegow"])
df["liczba drzwi"] = pd.to_numeric(df["liczba drzwi"])
df["kolor label"] = labelencoder.fit_transform(df["kolor"])

In [9]:
df.head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
0,21900,Toyota,RAV4,2003,218000,1794,Benzyna,125,Manualna,5.0,Niebieski,66,1090,0,2,175
2,32000,Mercedes-Benz,Klasa A,2013,110000,1595,Benzyna,122,Automatyczna,5.0,Inny kolor,46,814,0,0,157
3,34900,Toyota,AvensisGeneracjaIII (2009-),2012,325000,2231,Diesel,149,Manualna,5.0,Inny kolor,66,181,3,2,157
4,26500,Opel,AstraGeneracjaJ (2009-2019),2010,201000,1686,Diesel,110,Manualna,5.0,Srebrny,51,170,3,2,206
5,37000,Mercedes-Benz,Klasa C,2015,220000,2143,Diesel,204,Automatyczna,4.0,Inny kolor,46,821,3,0,157


In [10]:
df.shape

(49566, 16)

In [11]:
df = df.drop_duplicates()
df = df.dropna()

In [12]:
print(f"Number of unique cars: {df.shape[0]}")

Number of unique cars: 23602


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23602 entries, 0 to 49718
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cena                   23602 non-null  int64  
 1   marka                  23602 non-null  object 
 2   model                  23602 non-null  object 
 3   rocznik                23602 non-null  int64  
 4   przebieg               23602 non-null  int64  
 5   pojemnosc              23602 non-null  int64  
 6   rodzaj paliwa          23602 non-null  object 
 7   moc                    23602 non-null  int64  
 8   skrzynia biegow        23602 non-null  object 
 9   liczba drzwi           23602 non-null  float64
 10  kolor                  23602 non-null  object 
 11  marka label            23602 non-null  int32  
 12  model label            23602 non-null  int32  
 13  rodzaj paliwa label    23602 non-null  int32  
 14  skrzynia biegow label  23602 non-null  int32  
 15  ko

In [14]:
def find_outliers_limit(df,col):
    print(col)
    print('-'*50)
    #removing outliers
    q25, q75 = np.percentile(df[col], 25), np.percentile(df[col], 75)
    iqr = q75 - q25
    print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    print('Lower:',lower,' Upper:',upper)
    return lower,upper
def remove_outlier(df,col,upper,lower):
    # identify outliers
    outliers = [x for x in df[col] if x  == upper]
    print('Identified outliers: %d' % len(outliers))
    # remove outliers
    outliers_removed = [x for x in df[col] if x >= lower and x <= upper]
    print('Non-outlier observations: %d\n' % len(outliers_removed))
    final= np.where(df[col]>upper,upper,np.where(df[col]<lower,lower,df[col]))
    return final
outlier_cols=["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label", "cena"]
for col in outlier_cols:
    lower,upper=find_outliers_limit(df,col)
    df[col]=remove_outlier(df,col,upper,lower)

marka label
--------------------------------------------------
Percentiles: 25th=22.000, 75th=56.000, IQR=34.000
Lower: -29.0  Upper: 107.0
Identified outliers: 0
Non-outlier observations: 23602

model label
--------------------------------------------------
Percentiles: 25th=294.000, 75th=1149.000, IQR=855.000
Lower: -988.5  Upper: 2431.5
Identified outliers: 0
Non-outlier observations: 23602

rodzaj paliwa label
--------------------------------------------------
Percentiles: 25th=0.000, 75th=3.000, IQR=3.000
Lower: -4.5  Upper: 7.5
Identified outliers: 0
Non-outlier observations: 23602

kolor label
--------------------------------------------------
Percentiles: 25th=67.000, 75th=206.000, IQR=139.000
Lower: -141.5  Upper: 414.5
Identified outliers: 0
Non-outlier observations: 23602

rocznik
--------------------------------------------------
Percentiles: 25th=2006.000, 75th=2013.000, IQR=7.000
Lower: 1995.5  Upper: 2023.5
Identified outliers: 0
Non-outlier observations: 23541

przebieg

In [15]:
data = df[["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label", "cena"]]
X = data[["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]]
y = data["cena"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

regressor = XGBRegressor(objective ='reg:linear', colsample_bytree = 0.4, learning_rate = 0.1,
                max_depth = 5, alpha =10, n_estimators = 10)

regressor.fit(X_train, y_train)

[14:43:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=8,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [17]:
Y_pred_train = regressor.predict(X_train)
y_pred = regressor.predict(X_test)

In [18]:
def rmsle(y_pred,y_test) :
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    Acc = 1 - error
    return Acc

print("Accuracy attained on Training Set = ",rmsle(Y_pred_train, y_train))
print("Accuracy attained on Test Set = ",rmsle(y_pred,y_test))

Accuracy attained on Training Set =  0.7792615236756214
Accuracy attained on Test Set =  0.7802719039919879


In [19]:
lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

0.7216185434173492

In [20]:
df[df["model"] == "V50"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
215,29900.0,Volvo,V50,2012.0,198426.0,1560.0,Diesel,115.0,Manualna,5.0,Czarny,69.0,1415.0,3.0,2.0,67.0
528,10899.0,Volvo,V50,2004.0,239000.0,1998.0,Diesel,136.0,Manualna,5.0,Zielony,69.0,1415.0,3.0,2.0,265.0
547,13999.0,Volvo,V50,2005.0,225000.0,1998.0,Diesel,136.0,Manualna,5.0,Niebieski,69.0,1415.0,3.0,2.0,175.0
819,17900.0,Volvo,V50,2007.0,267000.0,1798.0,Benzyna,125.0,Manualna,5.0,Szary,69.0,1415.0,0.0,2.0,233.0
1269,12800.0,Volvo,V50,2005.0,320000.0,1998.0,Diesel,136.0,Manualna,5.0,Czarny,69.0,1415.0,3.0,2.0,67.0


In [21]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", "rocznik", 
 "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
volvo = [68.0, 1390.0, 3, 60, 2005, 
         260333, 1998, 140, 5, 2]
volvo = np.array(volvo).reshape((1,-1))
print(f"Price of volvo: {regressor.predict(volvo)[0].round()} zł")

Price of volvo: 13479.0 zł


In [22]:
df[df["model"] == "SX4"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
3495,22900.0,Suzuki,SX4,2007.0,188355.0,1586.0,Benzyna,107.0,Manualna,5.0,Czarny,64.0,1191.0,0.0,2.0,67.0
4237,31900.0,Suzuki,SX4,2012.0,143000.0,1586.0,Benzyna,120.0,Manualna,5.0,Szary,64.0,1191.0,0.0,2.0,233.0
4466,19900.0,Suzuki,SX4,2010.0,131551.0,1586.0,Benzyna,120.0,Manualna,5.0,Czarny,64.0,1191.0,0.0,2.0,67.0
5115,13500.0,Suzuki,SX4,2006.0,257000.0,1586.0,Benzyna,107.0,Manualna,5.0,Czerwony,64.0,1191.0,0.0,2.0,126.0
5139,22900.0,Suzuki,SX4,2009.0,199245.0,1586.0,Benzyna,107.0,Automatyczna,5.0,Czerwony,64.0,1191.0,0.0,0.0,126.0


In [23]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
suzuki = [61, 1119, 0, 111, 
          2007, 10700, 1586, 100, 5, 2]
suzuki = np.array(suzuki).reshape((1,-1))
print(f"Price of suzuki: {regressor.predict(suzuki)[0].round()} zł")

Price of suzuki: 17530.0 zł


In [24]:
df[df["model"] == "Fabia"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
107,33333.0,Škoda,Fabia,2015.0,27000.0,999.0,Benzyna,60.0,Manualna,5.0,Inny kolor,70.0,507.0,0.0,2.0,157.0
109,34700.0,Škoda,Fabia,2015.0,65000.0,999.0,Benzyna,60.0,Manualna,5.0,Niebieski,70.0,507.0,0.0,2.0,175.0
237,21900.0,Škoda,Fabia,2014.0,224257.0,1598.0,Diesel,90.0,Manualna,5.0,Srebrny,70.0,507.0,3.0,2.0,206.0
317,28900.0,Škoda,Fabia,2013.0,179000.0,1598.0,Diesel,105.0,Manualna,5.0,Czerwony,70.0,507.0,3.0,2.0,126.0
566,3900.0,Škoda,Fabia,2001.0,197789.0,1390.0,Benzyna,75.0,Manualna,5.0,Czarny,70.0,507.0,0.0,2.0,67.0


In [25]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
skoda = [63.0, 418.0, 0.0, 52.0, 
         2013, 185000, 1598.0, 100, 5, 2.0]
skoda = np.array(skoda).reshape((1,-1))
print(f"Price of skoda: {regressor.predict(skoda)[0].round()} zł")

Price of skoda: 19325.0 zł


In [26]:
df[df["model"] == "Coupe"]

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
670,16900.0,Hyundai,Coupe,2007.0,180000.0,1975.0,Benzyna,143.0,Manualna,5.0,Srebrny,27.0,438.0,0.0,2.0,206.0
2850,19900.0,Hyundai,Coupe,2008.0,171000.0,1975.0,Benzyna,143.0,Manualna,5.0,Czerwony,27.0,438.0,0.0,2.0,126.0
10695,19900.0,Fiat,Coupe,1997.0,235308.0,1998.0,Benzyna,147.0,Manualna,5.0,Niebieski,22.0,438.0,0.0,2.0,175.0
14986,15900.0,Hyundai,Coupe,2006.0,160000.0,1975.0,Benzyna,143.0,Manualna,5.0,Czarny,27.0,438.0,0.0,2.0,67.0
37946,7900.0,Hyundai,Coupe,2006.0,177756.0,1599.0,Benzyna,105.0,Manualna,5.0,Czarny,27.0,438.0,0.0,2.0,67.0
44743,11200.0,Renault,Coupe,2005.0,216000.0,1600.0,Benzyna,83.0,Manualna,5.0,Czarny,56.0,438.0,0.0,2.0,67.0
46441,9500.0,Hyundai,Coupe,2002.0,189000.0,1975.0,Benzyna,139.0,Manualna,5.0,Srebrny,27.0,438.0,0.0,2.0,206.0


In [27]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
hundai = [24, 281, 0, 75, 
          2007, 175000, 2000.0, 143, 3, 2]
hundai = np.array(hundai).reshape((1,-1))
print(f"Price of hundai: {regressor.predict(hundai)[0].round()} zł")

Price of hundai: 17075.0 zł


In [28]:
df[df["model"] == "508"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
112,26900.0,Peugeot,508,2011.0,260000.0,1560.0,Diesel,112.0,Manualna,5.0,Czarny,52.0,67.0,3.0,2.0,67.0
409,38888.0,Peugeot,508,2012.0,134975.0,1997.0,Diesel,163.0,Manualna,5.0,Czarny,52.0,67.0,3.0,2.0,67.0
466,63999.0,Peugeot,508,2015.0,154000.0,1997.0,Diesel,180.0,Automatyczna,5.0,Biały,52.0,67.0,3.0,0.0,6.0
553,66100.0,Peugeot,508,2015.0,140162.0,1997.0,Diesel,180.0,Automatyczna,5.0,Szary,52.0,67.0,3.0,0.0,233.0
698,36900.0,Peugeot,508,2011.0,199500.0,1997.0,Diesel,163.0,Automatyczna,5.0,Czarny,52.0,67.0,3.0,0.0,67.0


In [29]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
peugot = [51.0, 67.0, 3.0, 6.0, 
          2012, 168000, 1997, 204, 5, 0.0]
peugot = np.array(peugot).reshape((1,-1))
print(f"Price of peugot: {regressor.predict(peugot)[0].round()} zł")

Price of peugot: 31703.0 zł


In [30]:
by_brands = df.groupby("marka").count().sort_values("cena", ascending=False).head(20)
by_brands = pd.DataFrame(by_brands["cena"])
by_brands = by_brands.rename(columns={'cena':'ilosc aut'})
by_brands

,ilosc aut
marka,
Opel,2216
Audi,2029
Volkswagen,1884
BMW,1881
Ford,1833
Renault,1331
Mercedes-Benz,1313
Peugeot,1075
Toyota,1008
